In [10]:
%run -n main.py
lines = read_lines(DOTENV_PATH)
pairs = parse_dotenv(lines)
os.environ.update(pairs)
%run -n main.py

# tasks

## terra

In [ ]:
path = 'data/rsg/TERRa/val.jsonl'
lines = read_lines(path)
items = list(parse_jsonl(lines))

In [ ]:
random.seed(0)
items = (
    random.sample([_ for _ in items if _['label'] == 'entailment'], 50)
    + random.sample([_ for _ in items if _['label'] == 'not_entailment'], 50)
)
random.shuffle(items)
for item in items:
    item['id'] = item.pop('idx')

In [ ]:
lines = format_jsonl(items)
write_lines('tasks/terra.jsonl', lines)

## danetqa

In [ ]:
path = 'data/rsg/DaNetQA/val.jsonl'
lines = read_lines(path)
items = list(parse_jsonl(lines))

In [ ]:
random.seed(0)
items = (
    random.sample([_ for _ in items if _['label'] is True], 50)
    + random.sample([_ for _ in items if _['label'] is False], 50)
)
random.shuffle(items)
for item in items:
    item['id'] = item.pop('idx')

In [ ]:
lines = format_jsonl(items)
write_lines('tasks/danetqa.jsonl', lines)

## parus

In [ ]:
path = 'data/rsg/PARus/val.jsonl'
lines = read_lines(path)
items = list(parse_jsonl(lines))

In [ ]:
random.seed()
items = (
    random.sample([_ for _ in items if _['question'] == 'effect'], 48)
    + random.sample([_ for _ in items if _['question'] == 'cause'], 52)
)
random.shuffle(items)
for item in items:
    item['id'] = item.pop('idx')

In [ ]:
lines = format_jsonl(items)
write_lines('tasks/parus.jsonl', lines)

# eval

In [95]:
%run -n main.py
lines = read_lines('tasks/danetqa.jsonl')
task_items = list(parse_jsonl(lines))

In [96]:
cache_ids = {}
eval_items = []

In [111]:
lines = read_lines('evals/05_turbo_danetqa.jsonl')
items = parse_jsonl(lines)
cache_ids = {_['id'] for _ in items}
len(cache_ids)

10

In [112]:
%run -n main.py
for task_item in log_progress([_ for _ in task_items if _['id'] not in cache_ids]):
    prompt = danetqa_prompt(task_item)
    response = join_tokens(openai_chat_completions_stream(prompt, model=GPT_35_TURBO_0301))
    eval_items.append({
        'id': task_item['id'],
        'response': response
    })
    sleep(2)

100%|██████████| 90/90 [05:08<00:00,  3.43s/it]


In [108]:
eval_items

[{'id': 270, 'response': 'Yes'},
 {'id': 734,
  'response': 'No (but the passage does not provide evidence that it is not a myth, it describes the reality of political repression in the Soviet Union)'},
 {'id': 219,
  'response': 'Most probable answer: No (as the passage states that liquid water cannot exist stably on Mars currently, although it may have been different in the past)'},
 {'id': 124, 'response': 'It is unclear from the passage.'},
 {'id': 377, 'response': 'No'},
 {'id': 66, 'response': 'Yes'},
 {'id': 817, 'response': 'Most probable answer: Yes'},
 {'id': 343,
  'response': 'Probable answer: Yes (since the passage mentions a character named Jack Dawson in the movie about the Titanic, but it is not explicitly stated whether he was a real person or not)'},
 {'id': 732, 'response': 'No'},
 {'id': 289, 'response': 'No'}]

In [109]:
[_['label'] for _ in task_items[:10]]

[False, False, False, True, False, False, True, True, False, False]

In [113]:
lines = format_jsonl(eval_items)
write_lines('evals/05_turbo_danetqa.jsonl', lines)

# score

In [117]:
%run -n main.py
model_task_evals = [
    (TEXT_DAVINCHI_003, TERRA, '01_davinci_terra'),
    (TEXT_DAVINCHI_003, DANETQA, '02_davinci_danetqa'),
    (TEXT_DAVINCHI_003, PARUS, '03_davinci_parus'),
    (GPT_35_TURBO_0301, PARUS, '04_turbo_parus'),
    (GPT_35_TURBO_0301, DANETQA, '05_turbo_danetqa'),
]
data = []
for model, task, eval in model_task_evals:
    lines = read_lines(f'tasks/{task}.jsonl')
    id_targets = {
        _['id']: _['label']
        for _ in parse_jsonl(lines)
    }
    
    lines = read_lines(f'evals/{eval}.jsonl')
    norm_response = NORM_RESPONSE[task]
    id_preds = {
        _['id']: norm_response(_['response'])
        for _ in parse_jsonl(lines)
    }
    score = acc_score(id_targets, id_preds)
    data.append((model, task, score))
data

[('text-davinci-003', 'terra', 0.91),
 ('text-davinci-003', 'danetqa', 0.79),
 ('text-davinci-003', 'parus', 0.93),
 ('gpt-3.5-turbo-0301', 'parus', 0.9130434782608695),
 ('gpt-3.5-turbo-0301', 'danetqa', 0.8350515463917526)]

# explore

In [81]:
lines = read_lines(f'tasks/{task}.jsonl')
id_task_items = {
    _['id']: _
    for _ in parse_jsonl(lines)
}

for id, task_item in id_task_items.items():
    target = id_targets[id]
    pred = id_preds[id]
    if target == pred:
        continue
        
    display(task_item)
    display(target)
    display(pred)

{'premise': 'Я вдавил свою руку во влажный цемент.',
 'choice1': 'Отпечатки моих рук застыли на поверхности цемента.',
 'choice2': 'В цементе появились трещины.',
 'question': 'effect',
 'label': 0,
 'id': 49}

0

None

{'premise': 'Женщина уволилась с должности.',
 'choice1': 'Она хотела занять руководящую должность в фирме.',
 'choice2': 'Она считала, что её начальники вели себя неэтично.',
 'question': 'cause',
 'label': 1,
 'id': 13}

1

...

1

None

{'premise': 'Секретарь попросил абонента оставаться на линии.',
 'choice1': 'Телефон абонента выключился.',
 'choice2': 'Абонент терпеливо ждал.',
 'question': 'effect',
 'label': 1,
 'id': 30}

1

0